### 강의에서 소개된 파이썬 주요 기능
- scipy.spatial.Voronoi: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Voronoi.html
- numpy.hstack: https://numpy.org/doc/stable/reference/generated/numpy.hstack.html
- numpy.vstack: https://numpy.org/doc/stable/reference/generated/numpy.vstack.html
- matplotlib.pyplot.fill: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.fill.html

### 보로노이 분할 및 시각화

##### (1) 결합 데이터 불러오기

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial import Voronoi
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from src.plot_utils import draw_pitch

mpl.rcParams['animation.embed_limit'] = 100

In [ ]:
match_id = 1
file = f'data_metrica/Sample_Game_{match_id}/Sample_Game_{match_id}_IntegratedData.csv'
wide_traces = pd.read_csv(file, header=0, index_col=0)
wide_traces

##### (2) 보로노이 다이어그램(Voronoi diagram) 단순 산출 및 이미지 시각화

In [ ]:
data = wide_traces.loc[100]
players = [c[:-2] for c in data.dropna().keys() if c.endswith('_x') and c != 'ball_x']
x_cols = [f'{p}_x' for p in players]
y_cols = [f'{p}_y' for p in players]

points = pd.DataFrame(index=players)
points['x'] = data[x_cols].astype(float).values
points['y'] = data[y_cols].astype(float).values
points

In [ ]:
vor = Voronoi(points)
vor.vertices

In [ ]:
vor.regions

In [ ]:
region = vor.regions[3]
polygon = np.array([vor.vertices[i] for i in region])
polygon

In [ ]:
vor.point_region

In [ ]:
pitch_size = (104, 68)
fig, ax = draw_pitch(pitch='white', line='black', size_x=pitch_size[0]/10+8, size_y=pitch_size[1]/10+8)

for team, color in zip(['H', 'A'], ['r', 'b']):
    team_players = [p for p in players if p.startswith(team)]
    ax.scatter(*points.T[team_players].values, s=100, c=color, alpha=0.7, zorder=2)

    for player, point in points.loc[team_players].iterrows():
        if not np.isnan(point['x']):
            ax.text(point['x'] + 0.5, point['y'] + 0.5, int(player[1:3]), fontsize=13, color=color)

for i_point, (player, point) in enumerate(points.iterrows()):
    region = vor.regions[vor.point_region[i_point]]
    if not -1 in region:
        polygon = np.array([vor.vertices[i_vertex] for i_vertex in region])
        if player.startswith('H'):
            ax.fill(*polygon.T, alpha=0.5, ec='#863CAA', c='#FF5AE6', zorder=1)
        else:
            ax.fill(*polygon.T, alpha=0.5,  ec='#863CAA', c='#00B4FF', zorder=1)
            
ax.set_xlim(-40, pitch_size[0]+40)
ax.set_ylim(-40, pitch_size[1]+40)

plt.show()

##### (3) 대칭 이동된 선수 위치를 활용한 점유 공간 제한하기

In [ ]:
points = points[
    (points['x'] >= 0) & (points['x'] <= 104) &
    (points['y'] >= 0) & (points['y'] <= 68)
]
points_extended = np.hstack([
    np.vstack([points['x'], points['y']]),
    np.vstack([-points['x'], points['y']]),
    np.vstack([-points['x'] + pitch_size[0]*2, points['y']]),
    np.vstack([points['x'], -points['y']]),
    np.vstack([points['x'], -points['y'] + pitch_size[1]*2])
]).T
points_extended

In [ ]:
fig, ax = draw_pitch(pitch='white', line='black', size_x=pitch_size[0]/10+8, size_y=pitch_size[1]/10+8)
n_players = len(players)
n_home_players = len([p for p in players if p.startswith('H')])

for i in range(5):
    home_points = points_extended[n_players*i:n_players*i+n_home_players]
    away_points = points_extended[n_players*i+n_home_players:n_players*(i+1)]
    ax.scatter(*home_points.T, s=100, c='r', alpha=0.7, zorder=2)
    ax.scatter(*away_points.T, s=100, c='b', alpha=0.7, zorder=2)

ax.set_xlim(-110, pitch_size[0]+110)
ax.set_ylim(-90, pitch_size[1]+90)

plt.show()

##### (4) 경기장 내부 보로노이 다이어그램 산출 및 이미지 시각화

In [ ]:
def calc_voronoi(data):
    players = [c[:-2] for c in data.dropna().keys() if c.endswith('_x') and c != 'ball_x']
    points = pd.DataFrame(index=players)
    points['x'] = data[[f'{p}_x' for p in players]].astype(float).values
    points['y'] = data[[f'{p}_y' for p in players]].astype(float).values

    points = points[
        (points['x'] >= 0) & (points['x'] <= 104) &
        (points['y'] >= 0) & (points['y'] <= 68)
    ]
    points_extended = np.hstack([
        np.vstack([points['x'], points['y']]),
        np.vstack([-points['x'], points['y']]),
        np.vstack([-points['x'] + pitch_size[0]*2, points['y']]),
        np.vstack([points['x'], -points['y']]),
        np.vstack([points['x'], -points['y'] + pitch_size[1]*2])
    ]).T
    vor = Voronoi(points_extended)
    
    return points, vor

In [ ]:
points, vor = calc_voronoi(data)
fig, ax = draw_pitch(pitch='white', line='black')

for team, color in zip(['H', 'A'], ['r', 'b']):
    team_players = [p for p in players if p.startswith(team)]
    ax.scatter(*points.T[team_players].values, s=100, c=color, alpha=0.7, zorder=2)

    for player, point in points.loc[team_players].iterrows():
        if not np.isnan(point['x']):
            ax.text(point['x'] + 0.5, point['y'] + 0.5, int(player[1:3]), fontsize=13, color=color)

for i_point, point in enumerate(vor.points):
    region = vor.regions[vor.point_region[i_point]]
    if i_point < len(players) and not -1 in region:
        polygon = np.array([vor.vertices[i_vertex] for i_vertex in region])
        if i_point < len([p for p in players if p.startswith('H')]):
            ax.fill(*polygon.T, alpha=0.5, ec='#863CAA', c='#FF5AE6', zorder=1)
        else:
            ax.fill(*polygon.T, alpha=0.5,  ec='#863CAA', c='#00B4FF', zorder=1)

plt.show()

##### (5) 보로노이 다이어그램 애니메이션 시각화

In [ ]:
def plot_snapshot(
    data, figax=None, team_colors=('r', 'b'), annotate_players=True, annotate_events=True,
    show_velocities=True, show_sprints=False, show_voronoi=False
):
    if figax is None:
        fig, ax = draw_pitch(pitch='white', line='black')
    else:
        fig, ax = figax

    figobjs = []
    for team, color in zip(['H', 'A'], team_colors):
        team_players = [c[:-2] for c in data.dropna().keys() if c.startswith(team) and c.endswith('_x')]
        team_x_cols = [f'{p}_x' for p in team_players]
        team_y_cols = [f'{p}_y' for p in team_players]

        widths = pd.Series(0, index=team_players)
        if show_sprints:
            team_sprint_cols = [f'{p}_sprint' for p in team_players]
            widths = pd.Series(data[team_sprint_cols].clip(0, 1).values, index=team_players) * 4

        obj = ax.scatter(
            data[team_x_cols], data[team_y_cols], s=100, c=color,
            edgecolors='lime', linewidth=widths, zorder=2
        )
        figobjs.append(obj)

        if show_velocities:
            team_vx_cols = [f'{p}_vx' for p in team_players]
            team_vy_cols = [f'{p}_vy' for p in team_players]
            obj = ax.quiver(
                data[team_x_cols].astype(float), data[team_y_cols].astype(float),
                data[team_vx_cols].astype(float), data[team_vy_cols].astype(float),
                color=color, scale=8, scale_units='inches', width=0.002, alpha=0.7, zorder=2
            )
            figobjs.append(obj)
        
        if annotate_players:
            for x, y in zip(team_x_cols, team_y_cols):
                if not (np.isnan(data[x]) or np.isnan(data[y])):
                    obj = ax.text(data[x] + 0.5, data[y] + 0.5, int(x[1:3]), fontsize=13, color=color)
                    figobjs.append(obj)
    
    if show_voronoi:
        points, vor = calc_voronoi(data)
        
        for i_point, point in enumerate(vor.points):
            region = vor.regions[vor.point_region[i_point]]
            if i_point < len(points.index) and not -1 in region:
                polygon = np.array([vor.vertices[i_vertex] for i_vertex in region])
                if i_point < len([p for p in points.index if p.startswith('H')]):
                    obj, = ax.fill(*polygon.T, alpha=0.5, ec='#863CAA', c='#FF5AE6', zorder=1)
                else:
                    obj, = ax.fill(*polygon.T, alpha=0.5,  ec='#863CAA', c='#00B4FF', zorder=1)
                figobjs.append(obj)

    time_text = f"{int(data['time'] // 60):02d}:{data['time'] % 60:05.2f}"
    if annotate_events:
        if not pd.isnull(data['event_subtype']):
            event_text = f"{data['event_subtype']} by {data['event_player']}" 
        else:
            event_text = ''
        figobjs.append(ax.text(51, 69, time_text, fontsize=15, ha='right', va='bottom'))
        figobjs.append(ax.text(52, 69, '-', fontsize=15, ha='center', va='bottom'))
        figobjs.append(ax.text(53, 69, event_text, fontsize=15, ha='left', va='bottom'))
    else:
        figobjs.append(ax.text(52, 69, time_text, fontsize=15, ha='center', va='bottom'))
        
    obj = ax.scatter(data['ball_x'], data['ball_y'], s=80, color='w', edgecolors='k', zorder=3)
    figobjs.append(obj)
    
    return fig, ax, figobjs

In [ ]:
frame = 100
fig, ax, figobjs = plot_snapshot(wide_traces.loc[frame], show_voronoi=True)
plt.show()

In [ ]:
def save_clip(
    clip_traces, fname='test', fps=25, figax=None, team_colors=('r', 'b'),
    annotate_players=True, annotate_events=True, show_velocities=True, show_sprints=False, show_voronoi=False
):
    metadata = dict(title='Tracking Data', artist='Matplotlib', comment='Metrica tracking data clip')
    writer = animation.FFMpegWriter(fps=fps, metadata=metadata)

    if not os.path.exists('match_clips'):
        os.makedirs('match_clips')
    file = f'match_clips/{fname}.mp4'

    if figax is None:
        fig, ax = draw_pitch(pitch='white', line='black')
    else:
        fig, ax = figax
    fig.set_tight_layout(True)

    with writer.saving(fig, file, dpi=100):
        for i in clip_traces.index:
            frame_data = clip_traces.loc[i]
            fig, ax, figobjs = plot_snapshot(
                frame_data, (fig, ax), team_colors, annotate_players, annotate_events,
                show_velocities, show_sprints, show_voronoi
            )
            writer.grab_frame()

            for obj in figobjs:
                obj.remove()

    plt.clf()
    plt.close(fig)

In [ ]:
start_frame = 1000
end_frame = 1500
show_sprints = True
show_voronoi = True

fname_tokens = [f'game{match_id}', str(start_frame), str(end_frame)]
if show_sprints:
    fname_tokens.append('sprints')
if show_voronoi:
    fname_tokens.append('voronoi')
fname = '_'.join(fname_tokens)

clip_traces = wide_traces.loc[start_frame:end_frame]
save_clip(clip_traces, fname=f'{fname}.mp4', show_sprints=show_sprints, show_voronoi=show_voronoi)